In [119]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1.- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2.- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [172]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea


In [173]:
df_attack=attack[(attack["country"]=="usa") | (attack["country"]=="australia")|\
          (attack["country"]=="south africa") | (attack["country"]=="papua new guinea")|\
          (attack["country"]=="new zealand")]

In [176]:
df_attack.rename(columns={'country':'id_paises'}, inplace=True)

/var/folders/pq/y6t79q9j1z1gmlz1ggd9dck80000gn/T/ipykernel_36736/4288119105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.rename(columns={'country':'id_paises'}, inplace=True)


In [181]:
df_attack.sample(5)

,year,type,id_paises,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
2919,1983,Unprovoked,south africa,27.0,Tiger shark,Apr,N,M,-28.816624,24.991639,HS,1,0,0,1,0,0,2,0,7,6.313925
1563,2005,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
1241,2008,Provoked,south africa,24.0,Unespecific,May,N,M,-28.816624,24.991639,HS,1,0,0,1,0,0,3,9,4,5.936887
3324,1971,Unprovoked,usa,18.0,Tiger shark,Oct,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,2,12,7,5.080380
1314,2007,Unprovoked,usa,36.0,Unespecific,Sep,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,13,7,7.304807


In [200]:
df_attacks2=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
df_attacks2

/var/folders/pq/y6t79q9j1z1gmlz1ggd9dck80000gn/T/ipykernel_36736/2373186451.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_attacks2=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])


,year,age,latitud,longitud,fatal_N
,mean,mean,mean,mean,mean
id_paises,,,,,
australia,1945.205479,27.527397,-24.776109,134.755000,0.690259
new zealand,1941.134921,27.174603,-41.500083,172.834408,0.706349
papua new guinea,1914.460317,24.666667,-5.681607,144.248908,0.531746
south africa,1960.784965,26.159091,-28.816624,24.991639,0.709790
usa,1971.065544,26.842057,39.783730,-100.445882,0.821575


- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [124]:
df.sample(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
36,111,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 170, 'speed':...",23,2,8,1015,none,3,0,160,2,39.78373,-100.445882,USA
110,141,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 15, 'speed': ...",22,6,10,1024,none,3,0,20,3,-24.77610,134.755000,Australia


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [125]:
#el groupby es por pais

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [185]:
df['rh_profile']=df['rh_profile'].apply(ast.literal_eval)
rh = df['rh_profile'].apply(pd.Series)

for i in range(len(rh.columns)):
    rh_col0=rh[i].apply(pd.Series)
    nombre="rh"+ rh_col0["layer"][115]
    valores= list(rh_col0["rh"])
    df.insert(i, nombre, valores)


In [186]:
df=df.drop(['rh_profile'], axis=1)

df.sample(5)

,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
104,14,11,-1,-3,-1,1,0,-2,-3,-3,-3,-2,5,9,2,10,123,4,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 75, 'speed': ...",22,6,10,1023,rain,3,0,85,3,-24.776100,134.755000,Australia
146,15,13,13,14,15,12,12,8,-3,-4,-2,15,16,16,16,3,57,9,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 90, 'speed': ...",17,2,14,1015,rain,6,0,95,3,-28.816624,24.991639,South Africa
18,5,7,8,11,12,13,14,11,9,8,6,0,0,4,4,-3,57,2,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 45, 'speed': ...",27,2,3,1011,rain,3,0,65,2,39.783730,-100.445882,USA
154,15,15,14,11,13,15,15,15,7,13,14,16,16,16,16,-1,81,9,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 345, 'speed':...",17,2,15,1017,rain,6,0,0,2,-28.816624,24.991639,South Africa
245,11,15,13,-2,-3,-3,-3,-2,-2,-3,2,6,16,16,13,0,162,9,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 355, 'speed':...",11,10,8,1034,rain,3,0,355,3,-41.500083,172.834408,New Zealand


In [187]:
#Wind direction

df['wind_profile']=df['wind_profile'].apply(ast.literal_eval)
wind = df['wind_profile'].apply(pd.Series)

for i in range(len(wind.columns)):
    wind_col0=wind[i].apply(pd.Series)
    nombre="wind_"+ wind_col0["layer"][1]
    valores= list(wind_col0["direction"])
    df.insert(i, nombre, valores)


In [188]:
df=df.drop(['wind_profile'], axis=1)

df.sample(5)

,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
191,290,280,275,275,255,260,250,245,260,260,270,255,250,245,240,245,13,13,13,14,6,1,-4,-3,-3,-2,4,3,11,16,16,16,192,8,-9999,-9999,-9999,18,2,13,1020,rain,7,0,310,3,-28.816624,24.991639,South Africa
196,215,260,255,260,270,275,275,275,280,285,280,280,280,285,285,285,12,12,11,13,13,4,6,4,-2,-4,-4,-4,-3,-3,-4,-4,15,4,-9999,-9999,-9999,9,6,10,1017,rain,2,0,180,2,-41.500083,172.834408,New Zealand
162,320,325,345,350,345,320,325,325,330,335,325,320,305,305,305,300,10,12,16,16,15,15,14,15,14,15,13,15,4,-2,1,-1,105,9,-9999,-9999,-9999,17,6,12,1016,rain,7,0,335,3,-28.816624,24.991639,South Africa
85,140,135,115,155,160,170,175,180,200,210,195,190,195,200,205,225,12,12,14,-4,-4,-3,-3,-2,0,-1,1,0,-3,-2,1,1,66,8,-9999,-9999,-9999,21,6,9,1026,rain,3,0,140,4,-24.776100,134.755000,Australia
246,345,350,350,335,325,310,310,310,285,260,230,215,225,235,250,230,11,14,14,-2,-3,-3,-3,-2,-3,1,3,6,7,6,14,-1,165,9,-9999,-9999,-9999,10,10,8,1035,none,3,0,345,3,-41.500083,172.834408,New Zealand


In [189]:
df2=df.groupby('paises').agg(['mean'])
df2

/var/folders/pq/y6t79q9j1z1gmlz1ggd9dck80000gn/T/ipykernel_36736/3813191086.py:1: FutureWarning: ['prec_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df2=df.groupby('paises').agg(['mean'])


,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
paises,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,109.843750,139.921875,161.484375,180.468750,173.984375,183.984375,205.390625,219.375000,245.390625,252.265625,248.281250,249.609375,254.609375,250.937500,248.593750,253.593750,12.765625,10.906250,5.890625,1.328125,0.406250,0.203125,-0.296875,-1.015625,-1.515625,-1.781250,-2.078125,-1.031250,0.140625,2.093750,4.875000,1.390625,97.5,3.859375,-9999.0,-9999.0,-9999.0,21.328125,4.765625,10.171875,1022.859375,2.984375,0.0,114.843750,3.140625,-24.776100,134.755000
New Zealand,212.500000,214.296875,216.171875,226.875000,226.406250,228.203125,226.484375,228.437500,227.812500,227.812500,225.703125,217.421875,225.703125,236.640625,252.265625,262.890625,11.296875,13.375000,9.546875,3.687500,0.578125,-0.640625,-0.500000,-0.203125,-0.484375,0.296875,1.437500,0.640625,0.250000,0.671875,1.046875,-3.093750,97.5,7.781250,-9999.0,-9999.0,-9999.0,9.843750,9.468750,8.437500,1025.203125,2.593750,0.0,178.906250,3.046875,-41.500083,172.834408
Papua New Guinea,105.546875,102.031250,100.000000,99.765625,94.375000,93.828125,110.078125,114.140625,112.421875,120.781250,103.515625,197.500000,173.203125,199.843750,245.781250,279.140625,13.593750,11.890625,9.156250,6.093750,1.062500,-0.875000,-2.218750,-3.218750,-3.625000,-3.515625,-2.859375,-2.031250,-1.437500,-1.203125,-1.546875,-0.031250,97.5,2.109375,-9999.0,-9999.0,-9999.0,25.781250,1.609375,10.953125,1011.796875,2.281250,0.0,105.390625,3.703125,-5.681607,144.248908
South Africa,201.796875,217.812500,236.640625,288.671875,280.390625,263.515625,260.312500,270.000000,279.921875,282.187500,281.718750,276.796875,271.796875,267.734375,269.531250,265.937500,12.921875,11.984375,11.312500,8.890625,8.515625,8.953125,8.875000,8.421875,7.609375,6.968750,8.484375,10.062500,10.890625,8.890625,7.265625,3.671875,97.5,8.000000,-9999.0,-9999.0,-9999.0,16.609375,3.937500,12.484375,1018.046875,5.546875,0.0,163.125000,3.203125,-28.816624,24.991639
USA,174.062500,172.890625,193.437500,249.765625,292.500000,291.093750,286.406250,279.218750,277.109375,277.578125,278.203125,267.109375,248.906250,244.765625,245.625000,271.171875,8.546875,9.578125,9.265625,9.015625,8.078125,5.859375,3.656250,2.906250,3.234375,3.812500,4.718750,6.281250,6.968750,8.078125,9.140625,4.203125,97.5,4.281250,-9999.0,-9999.0,-9999.0,25.140625,0.140625,6.437500,1012.187500,2.984375,0.0,175.390625,2.296875,39.783730,-100.445882


In [190]:
df2=df2.reset_index()

In [191]:
df2.rename(columns={'paises':'id_paises'}, inplace=True)

In [202]:
def minuscula(col):
    try:
        col=col.lower()
        return col
    except:
        pass



In [203]:
df2["id_paises"]=df2["id_paises"].apply(minuscula)

In [204]:
merge_final=pd.merge(left=df2, right=df_attacks2, how='left', left_on='id_paises', right_on='id_paises')
merge_final

,id_paises,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_x,longitud_x,year,age,latitud_y,longitud_y,fatal_N
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,australia,109.843750,139.921875,161.484375,180.468750,173.984375,183.984375,205.390625,219.375000,245.390625,252.265625,248.281250,249.609375,254.609375,250.937500,248.593750,253.593750,12.765625,10.906250,5.890625,1.328125,0.406250,0.203125,-0.296875,-1.015625,-1.515625,-1.781250,-2.078125,-1.031250,0.140625,2.093750,4.875000,1.390625,97.5,3.859375,-9999.0,-9999.0,-9999.0,21.328125,4.765625,10.171875,1022.859375,2.984375,0.0,114.843750,3.140625,-24.776100,134.755000,1945.205479,27.527397,-24.776109,134.755000,0.690259
1,new zealand,212.500000,214.296875,216.171875,226.875000,226.406250,228.203125,226.484375,228.437500,227.812500,227.812500,225.703125,217.421875,225.703125,236.640625,252.265625,262.890625,11.296875,13.375000,9.546875,3.687500,0.578125,-0.640625,-0.500000,-0.203125,-0.484375,0.296875,1.437500,0.640625,0.250000,0.671875,1.046875,-3.093750,97.5,7.781250,-9999.0,-9999.0,-9999.0,9.843750,9.468750,8.437500,1025.203125,2.593750,0.0,178.906250,3.046875,-41.500083,172.834408,1941.134921,27.174603,-41.500083,172.834408,0.706349
2,papua new guinea,105.546875,102.031250,100.000000,99.765625,94.375000,93.828125,110.078125,114.140625,112.421875,120.781250,103.515625,197.500000,173.203125,199.843750,245.781250,279.140625,13.593750,11.890625,9.156250,6.093750,1.062500,-0.875000,-2.218750,-3.218750,-3.625000,-3.515625,-2.859375,-2.031250,-1.437500,-1.203125,-1.546875,-0.031250,97.5,2.109375,-9999.0,-9999.0,-9999.0,25.781250,1.609375,10.953125,1011.796875,2.281250,0.0,105.390625,3.703125,-5.681607,144.248908,1914.460317,24.666667,-5.681607,144.248908,0.531746
3,south africa,201.796875,217.812500,236.640625,288.671875,280.390625,263.515625,260.312500,270.000000,279.921875,282.187500,281.718750,276.796875,271.796875,267.734375,269.531250,265.937500,12.921875,11.984375,11.312500,8.890625,8.515625,8.953125,8.875000,8.421875,7.609375,6.968750,8.484375,10.062500,10.890625,8.890625,7.265625,3.671875,97.5,8.000000,-9999.0,-9999.0,-9999.0,16.609375,3.937500,12.484375,1018.046875,5.546875,0.0,163.125000,3.203125,-28.816624,24.991639,1960.784965,26.159091,-28.816624,24.991639,0.709790
4,usa,174.062500,172.890625,193.437500,249.765625,292.500000,291.093750,286.406250,279.218750,277.109375,277.578125,278.203125,267.109375,248.906250,244.765625,245.625000,271.171875,8.546875,9.578125,9.265625,9.015625,8.078125,5.859375,3.656250,2.906250,3.234375,3.812500,4.718750,6.281250,6.968750,8.078125,9.140625,4.203125,97.5,4.281250,-9999.0,-9999.0,-9999.0,25.140625,0.140625,6.437500,1012.187500,2.984375,0.0,175.390625,2.296875,39.783730,-100.445882,1971.065544,26.842057,39.783730,-100.445882,0.821575


- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [205]:
merge_final.to_csv('merge_clima_ataques.csv')